In [1]:
from src.cheffrey import *
import src.sugarcube as sc
from random import randint, sample, choice
from recipe_scrapers import scrape_me as scrape_recipe
from bs4 import BeautifulSoup
import requests
import os

%reload_ext autoreload
%autoreload 2

In [83]:
# import boto3
# import os
# s3 = boto3.client(
#     's3',
#     aws_access_key_id=os.getenv('AWS_ACCESS_KEY'),
#     aws_secret_access_key=os.getenv('AWS_SECRET_KEY'),
# )
# s3.download_file('cheffrey', 'recipes.yaml', './test.yaml')

In [2]:
recipes = load_yaml('recipes')

In [3]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [4]:
all_ingredients = [item for recipe in recipes.values() for item in recipe['Ingredients']]

In [14]:
ingredient = '1 (15-ounce) can chickpeas, with their liquid'
ing = ingredient.replace('-', ' ').split(',')[0]
ing = nlp(ing)
pos = [i.pos_ for i in ing]

In [30]:
parse_ingredient('1 (15-ounce) can chickpeas, with their liquid', nlp)

found (


(15.0, 'ounce', 'can chickpeas')

In [21]:
ingredient

'1 (15-ounce) can chickpeas, with their liquid'

In [108]:
import pandas as pd
# all_ingredients = all_ingredients[:500]
df = pd.DataFrame({'Ingredients': all_ingredients})
#not most efficient way. overhead of converting to series is memory intensive
def helper(x):
    try:
        i = parse_ingredient(x, nlp)
        if isinstance(i, sc.Ingredient):
            return pd.Series({'Amount': i.amount, 'Measure': i.amount.unit, 'Item': i.element})
        else:
            return pd.Series({'Amount': None, 'Measure': None, 'Item': i})
    except Exception as e:
        print(x)
        raise e

df[['Amount', 'Measure', 'Item']] = df['Ingredients'].apply(helper)

In [109]:
df.to_csv('parsed_ingredients.csv')